In [ ]:
!pip install pgmpy

In [ ]:
from pgmpy.factors.discrete import State
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling

In [ ]:
# Define the Bayesian Network
bn = BayesianNetwork([('T', 'F'), ('F', 'R'), ('T', 'R'), ('F', 'I'), ('C', 'I'), ('C', 'A')])

# Define the CPDs
cpd_T = TabularCPD('T', 2, [[0.92], [0.08]])
cpd_C = TabularCPD('C', 2, [[0.4], [0.6]])

cpd_F = TabularCPD('F', 2, [[0.998, 0.98], [0.002, 0.02]], ['T'], [2])
cpd_A = TabularCPD('A', 2, [[0.99, 0.9], [0.01, 0.1]], ['C'], [2])

cpd_R = TabularCPD('R', 2, [[0.99, 0.9, 0.1, 0.1], [0.01, 0.1, 0.9, 0.9]], ['T', 'F'], [2, 2])
cpd_I = TabularCPD('I', 2, [[0.999, 0.989, 0.99, 0.98], [0.001, 0.011, 0.01, 0.02]], ['C', 'F'], [2, 2])

# Add CPDs to the model
bn.add_cpds(cpd_T, cpd_C, cpd_F, cpd_A, cpd_R, cpd_I)

In [ ]:
def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

print_full(cpd_T)
print_full(cpd_C)
print_full(cpd_F)
print_full(cpd_A)
print_full(cpd_R)
print_full(cpd_I)

+------+------+
| T(0) | 0.92 |
+------+------+
| T(1) | 0.08 |
+------+------+
+------+-----+
| C(0) | 0.4 |
+------+-----+
| C(1) | 0.6 |
+------+-----+
+------+-------+------+
| T    | T(0)  | T(1) |
+------+-------+------+
| F(0) | 0.998 | 0.98 |
+------+-------+------+
| F(1) | 0.002 | 0.02 |
+------+-------+------+
+------+------+------+
| C    | C(0) | C(1) |
+------+------+------+
| A(0) | 0.99 | 0.9  |
+------+------+------+
| A(1) | 0.01 | 0.1  |
+------+------+------+
+------+------+------+------+------+
| T    | T(0) | T(0) | T(1) | T(1) |
+------+------+------+------+------+
| F    | F(0) | F(1) | F(0) | F(1) |
+------+------+------+------+------+
| R(0) | 0.99 | 0.9  | 0.1  | 0.1  |
+------+------+------+------+------+
| R(1) | 0.01 | 0.1  | 0.9  | 0.9  |
+------+------+------+------+------+
+------+-------+-------+------+------+
| C    | C(0)  | C(0)  | C(1) | C(1) |
+------+-------+-------+------+------+
| F    | F(0)  | F(1)  | F(0) | F(1) |
+------+-------+-------+---

In [ ]:
# Perform inference
inference = BayesianModelSampling(bn)

# Perform likelihood weighted sampling
samples = inference.likelihood_weighted_sample(evidence=[], size=100000, seed=1234)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
print(samples[0:10])

   T  F  R  I  C  A  _weight
0  0  0  0  0  0  0      1.0
1  0  0  0  0  1  0      1.0
2  0  0  0  0  1  0      1.0
3  0  0  0  0  0  0      1.0
4  0  0  0  1  1  0      1.0
5  0  0  0  0  0  0      1.0
6  0  0  0  0  1  0      1.0
7  0  0  0  0  1  0      1.0
8  1  0  1  0  0  0      1.0
9  0  0  0  0  0  0      1.0


In [ ]:
# Calculate the prior probability of a transaction being fraudulent
total_weight = samples['_weight'].sum()
prob_Fraudulent_1 = samples[samples['F'] == 1]['_weight'].sum() / total_weight
prob_Fraudulent_0 = samples[samples['F'] == 0]['_weight'].sum() / total_weight

print(f"Prior Probability of F being 0: {prob_Fraudulent_0}")
print(f"Prior Probability of F being 1: {prob_Fraudulent_1}")

Prior Probability of F being 0: 0.9968
Prior Probability of F being 1: 0.0032


In [ ]:
# Calculate the probability that the current transaction is a fraud given some factors
total_weight = samples[(samples['R'] == 0) & (samples['I'] == 1) & (samples['A'] == 0)]['_weight'].sum()
prob_Fraudulent_1 = samples[(samples['F'] == 1) & (samples['R'] == 0) & (samples['I'] == 1) & (samples['A'] == 0)]['_weight'].sum() / total_weight
prob_Fraudulent_0 = samples[(samples['F'] == 0) & (samples['R'] == 0) & (samples['I'] == 1) & (samples['A'] == 0)]['_weight'].sum() / total_weight

print(f"Prior Probability of F being 0: {prob_Fraudulent_0}")
print(f"Prior Probability of F being 1: {prob_Fraudulent_1}")

Prior Probability of F being 0: 0.9961013645224172
Prior Probability of F being 1: 0.003898635477582846


In [ ]:
# Calculate the probability that the current transaction is a fraud given some factors
total_weight = samples[(samples['R'] == 0) & (samples['I'] == 1) & (samples['A'] == 0) & (samples['T'] == 1)]['_weight'].sum()
prob_Fraudulent_1 = samples[(samples['F'] == 1) & (samples['R'] == 0) & (samples['I'] == 1) & (samples['A'] == 0) & (samples['T'] == 1)]['_weight'].sum() / total_weight
prob_Fraudulent_0 = samples[(samples['F'] == 0) & (samples['R'] == 0) & (samples['I'] == 1) & (samples['A'] == 0) & (samples['T'] == 1)]['_weight'].sum() / total_weight

print(f"Prior Probability of F being 0: {prob_Fraudulent_0}")
print(f"Prior Probability of F being 1: {prob_Fraudulent_1}")

Prior Probability of F being 0: 0.8333333333333334
Prior Probability of F being 1: 0.16666666666666666


In [ ]:
from pgmpy.inference import VariableElimination

In [ ]:
# Calculate the probability of a transaction being fraudulent
inference = VariableElimination(bn)
prob_Fraudulent = inference.query(variables=['F'])

print(f"Prior Probability of F being 1: {prob_Fraudulent}")

Prior Probability of F being 1: +------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.9966 |
+------+----------+
| F(1) |   0.0034 |
+------+----------+


In [ ]:
# Calculate the probability of a transaction being fraudulent
inference = VariableElimination(bn)
prob_Fraudulent = inference.query(variables=['F'], evidence={'R': 0, 'I': 1, 'A': 0})

print(f"Prior Probability of F being 1: {prob_Fraudulent}")

Prior Probability of F being 1: +------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.9948 |
+------+----------+
| F(1) |   0.0052 |
+------+----------+


In [ ]:
# Calculate the probability of a transaction being fraudulent
inference = VariableElimination(bn)
prob_Fraudulent = inference.query(variables=['F'], evidence={'R': 0, 'I': 1, 'A': 0, 'T': 1})

print(f"Prior Probability of F being 1: {prob_Fraudulent}")

Prior Probability of F being 1: +------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.9493 |
+------+----------+
| F(1) |   0.0507 |
+------+----------+


In [ ]:
sumOfDice = []

for i in range(1, 7):
  sumOfDiceI = []
  for j in range(1, 7):
    sumOfDiceJ = []
    for k in range(1, 7):
      sumOfDiceJ.append(i + j + k)
    sumOfDiceI.append(sumOfDiceJ)
  sumOfDice.append(sumOfDiceI)

from pprint import pprint
pprint(sumOfDice)

print("Total number of outcomes:", i * j * k)

def count_outcomes(target):
  counter = 0
  for i in range(1, 7):
    for j in range(1, 7):
      for k in range(1, 7):
        if sumOfDice[i-1][j-1][k-1] == target:
          counter+=1
  return counter

print("Outcomes for 2:", count_outcomes(2))
print("Outcomes for 7:", count_outcomes(7))
print("Outcomes for 12:", count_outcomes(12))
print("Outcomes for 15:", count_outcomes(15))

print("Outcomes for 9:", count_outcomes(9))
print("Outcomes for 18:", count_outcomes(18))

print("Outcomes for 5:", count_outcomes(5))
print("Outcomes for 9:", count_outcomes(9))
print("Outcomes for 10:", count_outcomes(10))
print("Outcomes for 13:", count_outcomes(13))

[[[3, 4, 5, 6, 7, 8],
  [4, 5, 6, 7, 8, 9],
  [5, 6, 7, 8, 9, 10],
  [6, 7, 8, 9, 10, 11],
  [7, 8, 9, 10, 11, 12],
  [8, 9, 10, 11, 12, 13]],
 [[4, 5, 6, 7, 8, 9],
  [5, 6, 7, 8, 9, 10],
  [6, 7, 8, 9, 10, 11],
  [7, 8, 9, 10, 11, 12],
  [8, 9, 10, 11, 12, 13],
  [9, 10, 11, 12, 13, 14]],
 [[5, 6, 7, 8, 9, 10],
  [6, 7, 8, 9, 10, 11],
  [7, 8, 9, 10, 11, 12],
  [8, 9, 10, 11, 12, 13],
  [9, 10, 11, 12, 13, 14],
  [10, 11, 12, 13, 14, 15]],
 [[6, 7, 8, 9, 10, 11],
  [7, 8, 9, 10, 11, 12],
  [8, 9, 10, 11, 12, 13],
  [9, 10, 11, 12, 13, 14],
  [10, 11, 12, 13, 14, 15],
  [11, 12, 13, 14, 15, 16]],
 [[7, 8, 9, 10, 11, 12],
  [8, 9, 10, 11, 12, 13],
  [9, 10, 11, 12, 13, 14],
  [10, 11, 12, 13, 14, 15],
  [11, 12, 13, 14, 15, 16],
  [12, 13, 14, 15, 16, 17]],
 [[8, 9, 10, 11, 12, 13],
  [9, 10, 11, 12, 13, 14],
  [10, 11, 12, 13, 14, 15],
  [11, 12, 13, 14, 15, 16],
  [12, 13, 14, 15, 16, 17],
  [13, 14, 15, 16, 17, 18]]]
Total number of outcomes: 216
Outcomes for 2: 0
Outcomes for 7: 15


In [ ]:
#q6 policy iteration

import copy

finding_customer = {1:0.2, 2: 0.6, 3: 0.4, 4:0.7}
destination={1:[2, 3, 4], 2:[1, 3], 3:[1, 4], 4:[1, 2]}
customer_destination={1:{2:0.4, 3: 0.2, 4: 0.4}, 2:{1:0.6, 3: 0.4}, 3:{1:0.6, 4:0.4}, 4:{1:0.4, 2:0.6}}
fare_cost = {1:{2:25, 3:12, 4:6}, 2:{1:10, 3:8}, 3:{1:15, 4:9}, 4:{1:11, 2:5}}
travel_cost = {1:{2:1, 3:1.5, 4:1.25}, 2:{1:1, 3:1.75, 4:1}, 3:{1:1.5, 2:1, 4:1.2}, 4:{1:1.25, 2:1, 3: 1}}
discount_factor = 0.95

v = {1:0, 2:0, 3:0, 4:0}
updating_v = {1:0, 2:0, 3:0, 4:0}

# best policy for each state
p = {1:4, 2:3, 3:2, 4:1}

for i in range(10):
  for l in range(1, 5):
    print(f"l:{l}, p[l]: {p[l]}")
    # determine value of policy
    curr_q = 0
    if p[l] == l:
      reward = 0
      for d in destination[p[l]]:
        print(f"reward += {finding_customer[p[l]]} * {customer_destination[p[l]][d]} * (({fare_cost[p[l]][d]} - {travel_cost[p[l]][d]})")
        reward += finding_customer[p[l]] * customer_destination[p[l]][d] * ((fare_cost[p[l]][d] - travel_cost[p[l]][d]))
      print(f"curr_q = {reward} + {discount_factor} * {v[p[l]]}")
      curr_q = reward + discount_factor * v[p[l]]
    else:
      print(f"curr_q = -{travel_cost[l][p[l]]} + {discount_factor} * {v[p[l]]}")
      curr_q = (-travel_cost[l][p[l]] + discount_factor * v[p[l]])

    max_q = curr_q
    best_policy = p[l]

    # check if a better action exists
    for a in range(1, 5):
      if (p[l] != a):
        print(f"a: {a}")
        q = 0
        if a == l:
            reward = 0
            for d in destination[a]:
              print(f"reward += {finding_customer[a]} * {customer_destination[a][d]} * (({fare_cost[a][d]} - {travel_cost[a][d]})")
              reward += finding_customer[a] * customer_destination[a][d] * ((fare_cost[a][d] - travel_cost[a][d]))
            print(f"q = {reward} + {discount_factor} * {v[a]}")
            q = reward + discount_factor * v[a]
        else:
            print(f"q = -{travel_cost[l][a]} + {discount_factor} * {v[a]}")
            q = (-travel_cost[l][a] + discount_factor * v[a])

        # update policy
        if q > max_q:
          max_q = q
          best_policy = a

      print(f"max_q: {max_q}")
      print(f"best_policy: {best_policy}")

    updating_v[l] = max_q
    p[l] = best_policy

  v = copy.deepcopy(updating_v)

  print(f"v: {v}")
  print(f"p: {p}")

l:1, p[l]: 4
curr_q = -1.25 + 0.95 * 0
a: 1
reward += 0.2 * 0.4 * ((25 - 1)
reward += 0.2 * 0.2 * ((12 - 1.5)
reward += 0.2 * 0.4 * ((6 - 1.25)
q = 2.72 + 0.95 * 0
max_q: 2.72
best_policy: 1
a: 2
q = -1 + 0.95 * 0
max_q: 2.72
best_policy: 1
a: 3
q = -1.5 + 0.95 * 0
max_q: 2.72
best_policy: 1
max_q: 2.72
best_policy: 1
l:2, p[l]: 3
curr_q = -1.75 + 0.95 * 0
a: 1
q = -1 + 0.95 * 0
max_q: -1.0
best_policy: 1
a: 2
reward += 0.6 * 0.6 * ((10 - 1)
reward += 0.6 * 0.4 * ((8 - 1.75)
q = 4.74 + 0.95 * 0
max_q: 4.74
best_policy: 2
max_q: 4.74
best_policy: 2
a: 4
q = -1 + 0.95 * 0
max_q: 4.74
best_policy: 2
l:3, p[l]: 2
curr_q = -1 + 0.95 * 0
a: 1
q = -1.5 + 0.95 * 0
max_q: -1.0
best_policy: 2
max_q: -1.0
best_policy: 2
a: 3
reward += 0.4 * 0.6 * ((15 - 1.5)
reward += 0.4 * 0.4 * ((9 - 1.2)
q = 4.4879999999999995 + 0.95 * 0
max_q: 4.4879999999999995
best_policy: 3
a: 4
q = -1.2 + 0.95 * 0
max_q: 4.4879999999999995
best_policy: 3
l:4, p[l]: 1
curr_q = -1.25 + 0.95 * 0
max_q: -1.25
best_policy: 1
a

In [ ]:
import copy

finding_customer = {1:0.2, 2: 0.6, 3: 0.4, 4:0.7}
destination={1:[2, 3, 4], 2:[1, 3], 3:[1, 4], 4:[1, 2]}
customer_destination={1:{2:0.4, 3: 0.2, 4: 0.4}, 2:{1:0.6, 3: 0.4}, 3:{1:0.6, 4:0.4}, 4:{1:0.4, 2:0.6}}
fare_cost = {1:{2:25, 3:12, 4:6}, 2:{1:10, 3:8}, 3:{1:15, 4:9}, 4:{1:11, 2:5}}
travel_cost = {1:{2:1, 3:1.5, 4:1.25}, 2:{1:1, 3:1.75, 4:1}, 3:{1:1.5, 2:1, 4:1.2}, 4:{1:1.25, 2:1, 3: 1}}
discount_factor = 0.95

v = {1:0, 2:0, 3:0, 4:0}
updating_v = {1:0, 2:0, 3:0, 4:0}

# best policy for each state
p = {1:4, 2:3, 3:2, 4:1}

# calculate reward
rewards = []

for l in range(1, 5):
  for a in range(1, 5):
    reward = 0
    if l == a:
      for d in destination[l]:
        reward += finding_customer[a] * customer_destination[a][d] * ((fare_cost[a][d] - travel_cost[a][d]))
    else:
      reward = -travel_cost[l][a]
    rewards.append(reward)

rewards = np.array(rewards)
rewards = rewards.reshape(4, 4)
print(rewards)

for i in range(10):
  for l in range(1, 5):
    # print(f"l:{l}, p[l]: {p[l]}")
    # determine value of policy
    curr_q = rewards[l-1][p[l]-1] + discount_factor * v[p[l]]

    max_q = curr_q
    best_policy = p[l]

    # check if a better action exists
    for a in range(1, 5):
      if (p[l] != a):
        # print(f"a: {a}")
        q = rewards[l-1][a-1] + discount_factor * v[a]

        # update policy
        if q > max_q:
          max_q = q
          best_policy = a

      # print(f"max_q: {max_q}")
      # print(f"best_policy: {best_policy}")

    updating_v[l] = max_q
    p[l] = best_policy

  v = copy.deepcopy(updating_v)

  print(f"v: {v}")
  print(f"p: {p}")

[[ 2.72  -1.    -1.5   -1.25 ]
 [-1.     4.74  -1.75  -1.   ]
 [-1.5   -1.     4.488 -1.2  ]
 [-1.25  -1.    -1.     4.41 ]]
v: {1: 2.72, 2: 4.74, 3: 4.4879999999999995, 4: 4.409999999999999}
p: {1: 1, 2: 2, 3: 3, 4: 4}
v: {1: 5.304, 2: 9.243, 3: 8.7516, 4: 8.599499999999999}
p: {1: 1, 2: 2, 3: 3, 4: 4}
v: {1: 7.780849999999999, 2: 13.52085, 3: 12.802019999999999, 4: 12.579524999999997}
p: {1: 2, 2: 2, 3: 3, 4: 4}
v: {1: 11.844807499999998, 2: 17.584807499999997, 3: 16.649918999999997, 4: 16.360548749999996}
p: {1: 2, 2: 2, 3: 3, 4: 4}
v: {1: 15.705567124999995, 2: 21.445567124999997, 3: 20.305423049999995, 4: 19.952521312499993}
p: {1: 2, 2: 2, 3: 3, 4: 4}
v: {1: 19.373288768749998, 2: 25.113288768749996, 3: 23.778151897499992, 4: 23.364895246874994}
p: {1: 2, 2: 2, 3: 3, 4: 4}
v: {1: 22.857624330312493, 2: 28.59762433031249, 3: 27.07724430262499, 4: 26.606650484531244}
p: {1: 2, 2: 2, 3: 3, 4: 4}
v: {1: 26.167743113796867, 2: 31.90774311379687, 3: 30.21138208749374, 4: 29.68631796030

In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 41.3 MB/s eta 0:00:00


In [ ]:
# q6 linear programming

import numpy as np
import pulp

finding_customer = {1:0.2, 2: 0.6, 3: 0.4, 4:0.7}
destination={1:[2, 3, 4], 2:[1, 3], 3:[1, 4], 4:[1, 2]}
customer_destination={1:{2:0.4, 3: 0.2, 4: 0.4}, 2:{1:0.6, 3: 0.4}, 3:{1:0.6, 4:0.4}, 4:{1:0.4, 2:0.6}}
fare_cost = {1:{2:25, 3:12, 4:6}, 2:{1:10, 3:8}, 3:{1:15, 4:9}, 4:{1:11, 2:5}}
travel_cost = {1:{2:1, 3:1.5, 4:1.25}, 2:{1:1, 3:1.75, 4:1}, 3:{1:1.5, 2:1, 4:1.2}, 4:{1:1.25, 2:1, 3: 1}}
discount_factor = 0.95

v = {1:0, 2:0, 3:0, 4:0}

rewards = []

# calculate reward
for l in range(1, 5):
  for a in range(1, 5):
    reward = 0
    if l == a:
      for d in destination[l]:
        reward += finding_customer[a] * customer_destination[a][d] * ((fare_cost[a][d] - travel_cost[a][d]))
    else:
      reward = -travel_cost[l][a]
    rewards.append(reward)

rewards = np.array(rewards)
rewards = rewards.reshape(4, 4)
print(rewards)

v = pulp.LpVariable.dicts("s", range(4)) # number of states
prob = pulp.LpProblem("taxi", pulp.LpMinimize)
prob += sum([v[i] for i in range(4)]) # defines the objective function
for l in range(4):
  prob += v[l] >= sum(rewards[l][a] + discount_factor * v[a] for a in range(4))
  prob += v[l] >= -25
  prob += v[l] <= 25

prob.solve()

print("Status:", pulp.LpStatus[prob.status])

V = np.zeros(4) # value function
for i in range(4):
    V[i] = v[i].varValue

# extract the optimal policy
pi_star = np.zeros((4), dtype=np.int64)
vduales = np.zeros((4, 4))
s = 0
a = 0
for name, c in list(prob.constraints.items()):
    vduales [s, a] = c.pi
    if a < 4 - 1:
        a = a + 1
    else:
        a = 0
        if s < 4 - 1:
            s = s + 1
        else:
            s = 0

print(vduales)
for s in range(4):
    pi_star[s] = np.argmax(vduales [s, :]) + 1

print(pi_star)

[[ 2.72  -1.    -1.5   -1.25 ]
 [-1.     4.74  -1.75  -1.   ]
 [-1.5   -1.     4.488 -1.2  ]
 [-1.25  -1.    -1.     4.41 ]]
Status: Optimal
[[0. 1. 0. 0.]
 [1. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]
[2 1 3 1]
